In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
MAIN_SITE = "https://eprocure.gov.in/"
TENDERS_2018_SITE = MAIN_SITE + "cppp/resultoftenders/byJWFieSViYnklY2J5JWRzZWxlY3RieSVlMjAxOGJ5JWZQdWJsaXNoZWQ=/page="
TENDERS_2017_SITE = MAIN_SITE + "cppp/resultoftenders/byJWFieSViYnklY2J5JWRzZWxlY3RieSVlMjAxN2J5JWZQdWJsaXNoZWQ=/page="
TENDERS_2016_SITE = MAIN_SITE + "cppp/resultoftenders/byJWFieSViYnklY2J5JWRzZWxlY3RieSVlMjAxNmJ5JWZQdWJsaXNoZWQ=/page="
TENDERS_2015_SITE = MAIN_SITE + "cppp/resultoftenders/byJWFieSViYnklY2J5JWRzZWxlY3RieSVlMjAxNWJ5JWZQdWJsaXNoZWQ=/page="
TENDERS_2014_SITE = MAIN_SITE + "cppp/resultoftenders/byJWFieSViYnklY2J5JWRzZWxlY3RieSVlMjAxNGJ5JWZQdWJsaXNoZWQ=/page="
TENDERS_2013_SITE = MAIN_SITE + "cppp/resultoftenders/byJWFieSViYnklY2J5JWRzZWxlY3RieSVlMjAxM2J5JWZQdWJsaXNoZWQ=/page="
WEBSITES = {2013: TENDERS_2013_SITE,
            2014: TENDERS_2014_SITE,
            2015: TENDERS_2015_SITE,
            2016: TENDERS_2016_SITE,
            2017: TENDERS_2017_SITE,
            2018: TENDERS_2018_SITE}
PAGES = {2013: 3819,
         2014: 7529,
         2015: 14930,
         2016: 19446,
         2017: 20368,
         2018: 13015}

In [6]:
WEBSITES = {2013: TENDERS_2013_SITE,
            2014: TENDERS_2014_SITE,
            2015: TENDERS_2015_SITE,
            2016: TENDERS_2016_SITE,
            2017: TENDERS_2017_SITE,
            2018: TENDERS_2018_SITE}
PAGES = {2013: 2,
         2014: 3,
         2015: 2,
         2016: 1,
         2017: 2,
         2018: 3}

def extract_links(table_html):
    links=[]
    link_tags = table_html.find_all('a')
    for link_tag in link_tags:
        links.append(MAIN_SITE + link_tag.get('href'))
    return(links)

def get_individual_tender_info(link):
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    table_indiv_html = soup.find('table', {"class": "viewtablebg"})
    individual_tender_raw=pd.read_html(str(table_indiv_html))[0]
    individual_tender = individual_tender_raw[[0,1]].transpose()
    individual_tender.columns = individual_tender.iloc[0] #Set first row as column headers
    individual_tender = individual_tender.reindex(individual_tender.index.drop(0)) #Drop first row after setting it as column headers
    individual_tender[individual_tender_raw[2][9]]=individual_tender_raw[3][9]
    return(individual_tender)
                                       
def individual_tenders_info(table_html):
    links = extract_links(table_html)
    individual_tenders_list=[]
    for link in links:
        individual_tenders_list.append(get_individual_tender_info(link))
    individual_tenders = pd.concat(individual_tenders_list)
    return(individual_tenders)

def make_tenders_table_page(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    table_html = soup.find('table', id="table")
    tenders_table = pd.read_html(str(table_html))[0]
    tenders_table.columns = tenders_table.iloc[0] #Set first row as column headers
    tenders_table = tenders_table.reindex(tenders_table.index.drop(0)) #Drop first row after setting it as column headers
    individual_tenders_table = individual_tenders_info(table_html)
    tenders_table = pd.concat([tenders_table.reset_index(drop=True), individual_tenders_table.reset_index(drop=True)], axis=1)
    return(tenders_table)

def make_tenders_table_year(year):
    final_year_table_list=[]
    for page in range(1,PAGES[year]+1):
        print(str(year) + ", page: " + str(page))
        url = WEBSITES[year] + str(page)
        final_year_table_list.append(make_tenders_table_page(url))
    final_year_table = pd.concat(final_year_table_list)
    return(final_year_table)
    
def make_tenders_table():
    final_table_list=[]
    for year, url in WEBSITES.items():
        curr_table=(make_tenders_table_year(year))
        curr_table["year"]=year
        final_table_list.append(curr_table)
    final_table=pd.concat(final_table_list)
    return(final_table)

final_table = make_tenders_table()

2013, page: 1
2013, page: 2


ValueError: No tables found

In [56]:
page = requests.get(WEBSITES[2018]+str(1))
soup = BeautifulSoup(page.content, 'html.parser')
table_html = soup.find('table', id="table")
tenders_table = pd.read_html(str(table_html))[0]
tenders_table.columns = tenders_table.iloc[0] #Set first row as column headers
tenders_table = tenders_table.reindex(tenders_table.index.drop(0)) #Drop first row after setting it as column headers
individual_tenders_table = individual_tenders_info(table_html)
#tenders_table = pd.concat([tenders_table, individual_tenders_table],axis=1)

In [4]:
page = requests.get('https://eprocure.gov.in//cppp/aocfullview/id%3D1122408')
page
soup = BeautifulSoup(page.content, 'html.parser')
soup
table_indiv = soup.find('table', {"class": "viewtablebg"})

In [70]:
pd.concat([tenders_table.reset_index(drop=True), individual_tenders_table.reset_index(drop=True)], axis=1)


,S.No.,AOC Date,e-Published Date,Title and Ref.No./Tender Id,Name of Organisation,Organisation Name :,Tender Ref. No. :,Tender Description :,Tender Document :,Tender Type :,Number of bids received :,Name of the selected bidder(s) :,Address of the selected bidder(s) :,Contract Value :,Published Date :,Date of Completion/Completion Period in Days :,Contract Date :
0,1.,11-Apr-2018 08:00 AM,28-Dec-2018 12:00 AM,SS Tubes & Fittings/ KKNPP/1&2/CMM/16061 / 24068,Nuclear Power Corporation of India Limited,Nuclear Power Corporation of India Limited,KKNPP/1&2/CMM/16061,SS Tubes & Fittings,https://npcil.etenders.in/manual_purchase_orde...,NaN,1,a to z industries,"88, issaji street, Noor Manzil 1st floor, masj...",101900,28-Dec-2018 12:00 AM,21-May-2018 08:00 AM,11-Apr-2018 08:00 AM
1,2.,26-Jun-2018 08:00 AM,27-Dec-2018 12:00 AM,LED LUMINARIES/ CMM/TOWNSHIP-ELE/56415 / 26197,Nuclear Power Corporation of India Limited,Nuclear Power Corporation of India Limited,CMM/TOWNSHIP-ELE/56415,LED LUMINARIES,https://npcil.etenders.in/manual_purchase_orde...,NaN,10,Ganit Star Engineering,"Agrawal Parisar, Nayta Mundla,Nemawar Road,Ind...",222600,27-Dec-2018 12:00 AM,10-Aug-2018 08:00 AM,26-Jun-2018 08:00 AM
2,3.,29-Jun-2018 08:00 AM,27-Dec-2018 12:00 AM,LED LUMINARIES/ CMM/TOWNSHIP-ELE/56415 / 26196,Nuclear Power Corporation of India Limited,Nuclear Power Corporation of India Limited,CMM/TOWNSHIP-ELE/56415,LED LUMINARIES,https://npcil.etenders.in/manual_purchase_orde...,NaN,10,JAIN INDUSTRIAL LIGHTING CORPORATION,B-70/21-22-23 DSIDC INDUSTRIAL COMPLEX LAWRENC...,521360,27-Dec-2018 12:00 AM,27-Sep-2018 08:00 AM,29-Jun-2018 08:00 AM
3,4.,05-May-2018 08:00 AM,26-Dec-2018 12:00 AM,TRANSPORTATION OF VERTICAL FLUX UNIT SPARE/ CM...,Nuclear Power Corporation of India Limited,Nuclear Power Corporation of India Limited,CMM/COMMON-CMM/HR/55804,TRANSPORTATION OF VERTICAL FLUX UNIT SPARE,https://npcil.etenders.in/manual_purchase_orde...,NaN,2,MFC TRANSPORT PVT.LTD,"1201,12th Floor, DLH Corporate Park, S. V. Roa...",490000,26-Dec-2018 12:00 AM,05-Nov-2019 08:00 AM,05-May-2018 08:00 AM
4,5.,27-Feb-2018 08:00 AM,18-Dec-2018 12:00 AM,PHOTO COPIER PAPER (WHITE COLOUR) A4 & A3 SIZE...,Nuclear Power Corporation of India Limited,Nuclear Power Corporation of India Limited,CMM/COMMON-CMM/HR/56376,PHOTO COPIER PAPER (WHITE COLOUR) A4 & A3 SIZE,https://npcil.etenders.in/manual_purchase_orde...,NaN,8,Mital stationery and printers,"Govt coloney, opp building no B 84, chetna mah...",722848,18-Dec-2018 12:00 AM,29-Mar-2018 08:00 AM,27-Feb-2018 08:00 AM
5,6.,09-May-2018 08:00 AM,05-Dec-2018 12:00 AM,Bolt and Nuts/ KKNPP/CMM/15430 / 24820,Nuclear Power Corporation of India Limited,Nuclear Power Corporation of India Limited,KKNPP/CMM/15430,Bolt and Nuts,https://npcil.etenders.in/manual_purchase_orde...,NaN,2,MOHANLAL AND COMPANY,"144, Dimond Building, Lohar Chawl,, Mumbai Mah...",1131.6,05-Dec-2018 12:00 AM,28-Jun-2018 08:00 AM,09-May-2018 08:00 AM
6,7.,04-May-2018 12:00 AM,05-Dec-2018 12:00 AM,HYDRO DESULPHURIZATION CATALYST FOR HGU III/ 3...,Mangalore Refinery and Petrochemicals Limited,Mangalore Refinery and Petrochemicals Limited,3000012115,HYDRO DESULPHURIZATION CATALYST FOR HGU III,NaN,Open,2,SUD-CHEMIE INDIA PVT LTD,NaN,8665800,05-Dec-2018 12:00 AM,NaN,04-May-2018 12:00 AM
7,8.,14-May-2018 08:00 AM,05-Dec-2018 12:00 AM,Bolt and Nuts/ KKNPP/CMM/15430 / 24821,Nuclear Power Corporation of India Limited,Nuclear Power Corporation of India Limited,KKNPP/CMM/15430,Bolt and Nuts,https://npcil.etenders.in/manual_purchase_orde...,NaN,2,YASHASWI ENGINEERS,"PLOT NO.09,M.S.F COMPOUND, MAJAGAON IND.ESTATE...",17953.74,05-Dec-2018 12:00 AM,22-Jun-2018 08:00 AM,14-May-2018 08:00 AM
8,9.,01-Dec-2018 09:00 AM,01-Dec-2018 09:00 AM,PAN BRG/ F5A1R12030 / F518A00060,BHEL Hyderabad,BHEL Hyderabad,F5A1R12030,PAN BRG,www.eprocure.gov.in/cppp/tendersearch,1,0,"UPPAL FERROCAST PRIVATE LTD.,",",,HYDERABAD",0,01-Dec-2018 09:00 AM,16-Feb-2019 09:00 AM,01-Dec-2018 09:00 AM
9,10.,01-Dec-2018 09:00 AM,01-D

In [163]:
link = table.find_all('a')
link[0].get('href')
#rows[3].td.contents[0].strip()

'/cppp/aocfullview/id%3D1122408'

In [37]:
test[1].as_matrix(columns=test.columns[:])

array(['Nuclear Power Corporation of India Limited', 'KKNPP/1&2/CMM/16061',
       'SS Tubes & Fittings',
       'https://npcil.etenders.in/manual_purchase_orders/view_vendor_manual_po/20184/24068'], dtype=object)